In [12]:
#Import nessesary libraries

import pandas as pd
import sqlite3
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.error import URLError
import numpy as np
from itertools import permutations
from collections import OrderedDict
from datetime import timedelta

# pip install geopy #This is used to calculate distance based on lat and long... no need to reinvent the wheel...

from geopy.distance import great_circle

import matplotlib.pyplot as plt
%matplotlib inline



In [5]:
#Read in top airport data from supplied files
#combine the two data frames into one
path = 'hw_5_data/'
#db_name = 'airports2.db'
db_name = 'airport_db'
top_airpots = pd.read_csv(path+'top_airports.csv' )
airport_info = pd.read_csv(path+'ICAO_airports.csv' )

airport_info['ICAO'] = airport_info['ident']
combined_airport_info = top_airpots.merge(airport_info, on='ICAO')

combined_airport_info

,City,FAA,IATA,ICAO,Airport,Role,Enplanements,id,ident,type,...,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,Atlanta,ATL,ATL,KATL,Hartsfield-Jackson Atlanta International Airport,P-L,43130585,3384,KATL,large_airport,...,US,US-GA,Atlanta,yes,KATL,ATL,ATL,http://www.atlanta-airport.com/,http://en.wikipedia.org/wiki/Hartsfield-Jackso...,NaN
1,Chicago,ORD,ORD,KORD,Chicago O'Hare International Airport,P-L,32171831,3754,KORD,large_airport,...,US,US-IL,Chicago,yes,KORD,ORD,ORD,http://www.flychicago.com/Ohare/OhareHomepage....,http://en.wikipedia.org/wiki/O'Hare_Internatio...,"CHI, Orchard Place"
2,Los Angeles,LAX,LAX,KLAX,Los Angeles International Airport,P-L,30528737,3632,KLAX,large_airport,...,US,US-CA,Los Angeles,yes,KLAX,LAX,LAX,http://www.iflylax.com/,http://en.wikipedia.org/wiki/Los_Angeles_Inter...,NaN
3,Dallas-Fort Worth,DFW,DFW,KDFW,Dallas/Fort Worth International Airport,P-L,27100656,3488,KDFW,large_airport,...,US,US-TX,Dallas-Fort Worth,yes,KDFW,DFW,DFW,http://www.dfwairport.com/,http://en.wikipedia.org/wiki/Dallas-Fort_Worth...,QDF
4,Denver,DEN,DEN,KDEN,Denver International Airport,P-L,25241962,3486,KDEN,large_airport,...,US,US-CO,Denver,yes,KDEN,DEN,DEN,http://www.flydenver.com/,http://en.wikipedia.org/wiki/Denver_Internatio...,NaN
5,New York,JFK,JFK,KJFK,John F. Kennedy International Airport,P-L,22934047,3622,KJFK,large_airport,...,US,US-NY,New York,yes,KJFK,JFK,JFK,http://www.panynj.gov/CommutingTravel/airports...,http://en.wikipedia.org/wiki/John_F._Kennedy_I...,"Manhattan, New York City, NYC, Idlewild"
6,San Francisco,SFO,SFO,KSFO,San Francisco International Airport,P-L,20038679,3878,KSFO,large_airport,...,US,US-CA,San Francisco,yes,KSFO,SFO,SFO,http://www.flysfo.com/,http://en.wikipedia.org/wiki/San_Francisco_Int...,"QSF, QBA"
7,Houston,IAH,IAH,KIAH,George Bush Intercontinental Airport,P-L,19528631,3604,KIAH,large_airport,...,US,US-TX,Houston,yes,KIAH,IAH,IAH,http://www.fly2houston.com/iah,http://en.wikipedia.org/wiki/George_Bush_Inter...,QHO
8,Las Vegas,LAS,LAS,KLAS,McCarran International Airport,P-L,18996738,3631,KLAS,large_airport,...,US,US-NV,Las Vegas,yes,KLAS,LAS,LAS,http://www.mccarran.com/,http://en.wikipedia.org/wiki/McCarran_Internat...,NaN
9,Phoenix,PHX,PHX,KPHX,Phoenix Sky Harbor International Airport,P-L,18907171,3772,KPHX,large_airport,...,US,US-AZ,Phoenix,yes,KPHX,PHX,PHX,http://phoenix.gov/skyharborairport/,http://en.wikipedia.org/wiki/Phoenix_Sky_Harbo...,NaN


## Make SQL tables

In [6]:
connection = sqlite3.connect(path+db_name)
cursor = connection.cursor()

#Create a table for the airport info and a table for the corresponding weather
sql_info_cmd = """CREATE TABLE info (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ICAO TEXT,
    name TEXT,
    city TEXT,
    latitude_deg FLOAT,
    longitude_deg FLOAT,
    elevation FLOAT,
    enplanements INT)"""


sql_weather_cmd = """CREATE TABLE weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ICAO TEXT,
    date DATE,
    temp_high FLOAT,
    temp_ave FLOAT,
    temp_low FLOAT,
    humidity_ave FLOAT,
    precipitation FLOAT)"""

cursor.execute(sql_info_cmd)
cursor.execute(sql_weather_cmd)
connection.commit()

## Helper Functions to deal with formating dates for data frames

In [7]:
# Helper functions to format dates and get date range to request data

def month_to_int(string):
    month_dict ={'Jan' : 1,
                 'Feb' : 2,
                 'Mar' : 3,
                 'Apr' : 4,
                 'May' : 5,
                 'Jun' : 6,
                 'Jul' : 7,
                 'Aug' : 8,
                 'Sep' : 9,
                 'Oct' : 10,
                 'Nov' : 11,
                 'Dec' : 12}
    return month_dict[string]

def month_to_str(int):
    months =['Jan' ,'Feb' ,'Mar' ,'Apr' ,'May' ,'Jun' ,'Jul' ,'Aug' ,'Sep' ,'Oct' ,'Nov' ,'Dec' ]
    return months[int]


def strip_date(date):
    date_temp = date.split('/')
    year = date_temp[0]
    month = date_temp[1]
    day = date_temp[2]
    
    return year,month,day

def inc_by_year(date):
    '''
    Helper function to incriment a given date by one year,
    expects to be in format %Y/%m/%d
    '''
    date_temp = date.split('/')
    year_inc = str(int(date_temp[0])+1)
    new_date = '{}/{}/{}'.format(year_inc, date_temp[1], date_temp[2])
    return new_date

def inc_by_day(date):
    '''
    Helper function to incriment a given date by one day,
    expects to be in format %Y/%m/%d
    '''
    date_temp = date.split('/')
    day_inc = str(int(date_temp[2])+1)
    new_date = '{}/{}/{}'.format(date_temp[0], date_temp[1], day_inc)
    return new_date


def get_date_range_to_query(date_start,date_end):
    '''
    Weatherunderground historical data only returns a years worth of data at a time,
    this function takes the users defined date range, and returns a list of start and end dates divided
    into year long chunks. 
    '''
    if date_end == 'today':
        today = datetime.now()
        date_end = '{}/{}/{}'.format(today.year, today.month, today.day)
    
    date_range = list()
    dates_temp_list = list()
    
    dates_temp_list.append(date_start)
    
    date_delta = (datetime.strptime(date_end, '%Y/%m/%d') - datetime.strptime(date_start, '%Y/%m/%d'))
    date_delta_years = int(date_delta.total_seconds()/(60*60*24*365.2425))
    
    for iyear in range(date_delta_years):
        temp_date = inc_by_year(dates_temp_list[iyear])
        dates_temp_list.append(temp_date)
        
    lgc_first_interation = True    
    
    for idate in range(len(dates_temp_list)):
        if lgc_first_interation:
            try:
                date_range.append([dates_temp_list[idate],dates_temp_list[idate+1]])
            except:
                date_range.append([dates_temp_list[idate],date_end])           
        else:
            try:
                date_range.append([inc_by_day(dates_temp_list[idate]),dates_temp_list[idate+1]])
            except:
                date_range.append([dates_temp_list[idate],date_end])
        lgc_first_interation = False  
    
    return date_range


#get_date_range_to_query(date_start,date_end='today')
    
    
    

# Scrape data from weather underground


In [8]:
def scrape_data(airport_code, date_start,date_end,data_needed,data_list):
    '''
    This function getting the info from WU for a specified start and end date, for a given airport code.
    'data_needed' is an ordered ditcionary that gives tells what columns of data to keep. This function expects 
    a range of dates of a year or less. This requirement will be taken care of by the program that calls it. 
    
    '''
    start_year, start_month, start_day = strip_date(date_start)
    end_year, end_month, end_day = strip_date(date_end)
    
    month_str = month_to_str(int(start_month))
    current_year = int(start_year)
    current_month_temp = int(start_month)
    
    url = url = 'https://www.wunderground.com/history/airport/{}/{}/CustomHistory.html?dayend={}&monthend={}&yearend={}'.format(
            airport_code, date_start, end_day, end_month, end_year)
    
    response = urlopen(url)
    html = response.read()
    response.close()
    
    # Use Beautiful soup 
    soup = BeautifulSoup(html, "html.parser")
    
    # Grab the second table
    table = soup.findAll('table')[1]
    results = list()

    for iline, line in enumerate(table.findAll('tr')):
        #we need to get ignore the header. All the headers have the text 'high' 'low' 'ave' ..etc. so 
        #if we just check that one of these words is not in the line we are looking at. If it is, then we grab
        #the month name from it, then we will skip the header

        values_per_line = list()
        if 'high' in line.text:
            month_temp = line.text[0]
            for ivalue, value in enumerate(line.findAll('td')):
                if ivalue == 0:
                    month_str = ''.join([word.rstrip() for word in value.findAll(text=True)])
                    current_month = month_to_int(month_str)
            #print(month_str)
        if 'high' not in line.text:

            #print(iline)    
            #itterate through all the values in the line
            for ivalue, value in enumerate(line.findAll('td')):

                #print(value)
                if ivalue in data_needed.values():
                    
                    stripped_vals = ''.join([word.rstrip() for word in value.findAll(text=True)])
                    #print(stripped_vals)
                    values_per_line.append(stripped_vals)
            

        if values_per_line != []: #make sure there is data in the list
            if current_month < int(current_month_temp):
                current_year += 1
                current_month_temp = current_month
            if current_month == start_month:
                if values_per_line[0] < int(start_day):
                    current_year += 1
                    current_month_temp = current_month
                    
            temp_date =  '{}-{}-{}'.format(str(current_year),str(current_month),str(values_per_line[0]))
            values_per_line[0] = temp_date
            #temp_date = '{}/{}/{}'.format(start_year,start_month,values_per_line[0])
            #values_per_line[0]
            
            
            
            data_list.append(values_per_line)
    
    return data_list

def get_weather_data(airport_code,date_start='2008/3/11',date_end = 'today'):
    
    data_needed = OrderedDict({'date': 0, 'temp_high': 1, 'temp_ave': 2, 'temp_low': 3,
                                    'humidity_ave': 8, 'precipitation': 19})
    
    date_range = get_date_range_to_query(date_start,date_end)
    data_list = []
    for idate in range(len(date_range)):
        while True:
            try:
                scrape_data(airport_code\
                                   , date_range[idate][0],date_range[idate][1],data_needed,data_list)
                break
            except URLError:
                print('Connection to WeatherUnderground broken')
                pass
        
            
    data_df = pd.DataFrame(data_list, columns=data_needed.keys())
    for key in data_needed.keys():
        if key != 'date':
            data_df[key] = pd.to_numeric(data_df[key], errors='coerce')
    data_df = data_df.fillna(0)
    
    return data_df
    

    

    

   

In [15]:
toot = get_weather_data('KATL')

In [23]:
toot.head()

,date,temp_high,temp_ave,temp_low,humidity_ave,precipitation
0,2008-3-11,56,51,45,64,0.00
1,2008-3-12,68,53,37,53,0.00
2,2008-3-13,72,61,49,42,0.00
3,2008-3-14,63,58,52,72,0.76
4,2008-3-15,75,65,55,82,0.40


### Populate airport info table

In [9]:
print('Populating info table...')
for i, line in combined_airport_info.iterrows():
    print('Iteration: ', i)
    data_per_line = (line['ICAO'], line['Airport'], line['City'], line['latitude_deg'], line['longitude_deg'],
                  line['elevation_ft'], line['Enplanements'])
    cursor.execute('INSERT INTO info' + '(ICAO, name, city, latitude_deg, longitude_deg, elevation,\
                    enplanements) ' + 'VALUES (?, ?, ?, ?, ?, ?, ?)', data_per_line)
connection.commit()    

Populating info table...
Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49


### Populate airport weather table

In [26]:
print('Populating weather table...')
#for i, line in combined_airport_info.iterrows():
#line['ICAO']

    #print('Iteration: ', i)
    #if i == 0:
    
weather_data = toot #get_weather_data(airport_code = line['ICAO'])
for irow, weatherRow in weather_data.iterrows():
    data =  ('KSNA', weatherRow['date'], weatherRow['temp_high'], 
            weatherRow['temp_ave'], weatherRow['temp_low'], weatherRow['humidity_ave'], weatherRow['precipitation'])
    cursor.execute('INSERT INTO weather' + '(ICAO, date, temp_high, temp_ave, temp_low, humidity_ave, precipitation) ' + \
                    'VALUES (?, ?, ?, ?, ?, ? , ?)', data)
connection.commit()    

Populating weather table...


NameError: name 'toot' is not defined

# Analysis Section

In [4]:
#Helper function to get data from a given database

def get_data_from_db(db_name, path, sql_cmd):
        connection = sqlite3.connect(path+db_name)
        cursor = connection.cursor()
        connection.row_factory = lambda cursor, row: row[0]
        c = connection.cursor()
        data = c.execute(sql_cmd).fetchall()
        connection.close()
        return data
    

### Analysis computation code

In [18]:


#Create dictionary to hold all the correlation data
airport_pairs_dict = {}
column_names = ['Airport_pairs','highT_1','highT_3','highT_7',\
                'precip_1','precip_3','precip_7','dist','long_delta' ]
for key in column_names: 
    airport_pairs_dict[key] = list()
    
    

###Get airport codes from database
sql_cmd = "SELECT ICAO FROM airport_info"
airport_code = get_data_from_db(db_name,path, sql_cmd)



##### temporary code to eliminate bad files in database ###
bad_airport = np.array(['KIAH','KLGA','KMSY','KIND'])
airport_code = np.asarray(airport_code)
cuts = np.ones(shape=airport_code.shape, dtype=bool)
bad_indx = np.where(airport_code == 'KIAH')#np.array([word for word in bad_airport]))
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KLGA')#[word for word in bad_airport])
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KMSY')#[word for word in bad_airport])
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KIND')#[word for word in bad_airport])
cuts[bad_indx] = 0


### calculate the correlations for all permutations of airports
for combo in permutations(airport_code[cuts],2):
    
    
    sql_cmd_highT_1 = "SELECT high_temp FROM airport_weather where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_highT_2 = "SELECT high_temp FROM airport_weather where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_precip_1 = "SELECT precipitation FROM airport_weather where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_precip_2 = "SELECT precipitation FROM airport_weather where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_lat_1 = "SELECT latitude_deg FROM airport_info where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_lat_2 = "SELECT latitude_deg FROM airport_info where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_long_1 = "SELECT longitude_deg FROM airport_info where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_long_2 = "SELECT longitude_deg FROM airport_info where ICAO = " + '"' + str(combo[1] + '"')
    

    high_temp_1 = get_data_from_db(db_name,path,sql_cmd_highT_1)
    high_temp_2 = get_data_from_db(db_name,path,sql_cmd_highT_2)
    
    precip_1 = get_data_from_db(db_name,path,sql_cmd_precip_1)
    precip_2 = get_data_from_db(db_name,path,sql_cmd_precip_2)
    
    lat_1 = get_data_from_db(db_name,path,sql_cmd_lat_1)[0]
    lat_2 = get_data_from_db(db_name,path,sql_cmd_lat_2)[0]
    
    long_1 = get_data_from_db(db_name,path,sql_cmd_long_1)[0]
    long_2 = get_data_from_db(db_name,path,sql_cmd_long_2)[0]
    
    #when calculating correlations, shift one array by 1,3,and 7 days to get
    #the 1,3, and 7 day correlations
    one_highT_coeff   = np.corrcoef(high_temp_1[:-1],high_temp_2[1:])[0,1]
    three_highT_coeff = np.corrcoef(high_temp_1[:-3],high_temp_2[3:])[0,1]
    seven_highT_coeff = np.corrcoef(high_temp_1[:-7],high_temp_2[7:])[0,1]
    
    one_precip_coeff   = np.corrcoef(precip_1[:-1],precip_2[1:])[0,1]
    three_precip_coeff = np.corrcoef(precip_1[:-3],precip_2[3:])[0,1]
    seven_precip_coeff = np.corrcoef(precip_1[:-7],precip_2[7:])[0,1]
    
    long_delta = long_2-long_1 #difference in longitude
    
    distance = great_circle((lat_1,long_1),(lat_2,long_2)).miles #absolute distance between cities
    
    #append each value to our dictionary
    airport_pairs_dict['Airport_pairs'].append(combo)
    airport_pairs_dict['highT_1'].append(one_highT_coeff)
    airport_pairs_dict['highT_3'].append(three_highT_coeff)
    airport_pairs_dict['highT_7'].append(seven_highT_coeff)
    airport_pairs_dict['precip_1'].append(one_precip_coeff)
    airport_pairs_dict['precip_3'].append(three_precip_coeff)
    airport_pairs_dict['precip_7'].append(seven_precip_coeff)
    airport_pairs_dict['dist'].append(distance)
    airport_pairs_dict['long_delta'].append(long_delta)
    

#turn data dictonary into pandas df, setting the city pair as the index
pairs_df = pd.DataFrame.from_dict(airport_pairs_dict).set_index('Airport_pairs') 

    
    

    
    

### Plotting

In [5]:
def plot_results(df):
    x_val = ['long_delta','dist']
    y_val = ['highT_1','highT_3','highT_7','precip_1','precip_3','precip_7']
    
    fig, ax = plt.subplots(1, len(x_val), figsize = (12, 3 * len(x_val)))
    
    for ii in range(len(x_val)):
        for jj in range(len(y_val)):
            
            ax[ii].set_xlabel(x_val[ii])
            ax[ii].set_ylabel('Correlation coefficien')
            
            df_sort = df.sort_values(by = y_val[jj], ascending=False)[:10]
            
            ax[ii].plot(df_sort[x_val[ii]], df_sort[y_val[jj]], '0', label = y_val[jj])
            ax[ii].legend(loc = 'best')
            
    plt.tight_layout()
    
    
    
    
    
    
    
    
    
    
plot_results(pairs_df)


NameError: name 'pairs_df' is not defined